In [8]:
import pandas as pd
from datetime import timedelta
from sklearn.impute import SimpleImputer

In [9]:
file_path = 'Sample logic test-recon.xlsx'
excel_data = pd.ExcelFile(file_path)

In [15]:
sheet_26as_df = pd.read_excel(file_path, sheet_name='26as')
sr_df = pd.read_excel(file_path, sheet_name='Sr')

In [16]:
print("26as sheet columns:", sheet_26as_df.columns)
print("Sr sheet columns:", sr_df.columns)

26as sheet columns: Index([ 'Unnamed: 0',  'Unnamed: 1',  'Unnamed: 2',  'Unnamed: 3',
        'Unnamed: 4',  'Unnamed: 5',  'Unnamed: 6',     141560.55,
                2124,  'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29'],
      dtype='object')
Sr sheet columns: Index([              'S.No',      'Customer Name',         'TAN Number',
           'Document Numer',              'Batch',             'Period',
            'Taxable Value',     'GST Tax Amount',      'Invoice Value',
             'Posting Date',      'Document Date',  'Clearing Document',
         'Bank Rcpt amount',   'Bank Credit Date',         'TDS Amount',
                 'TDS Date',         'PAN Number',              'GST

In [17]:
sheet_26as_df.columns = sheet_26as_df.iloc[0].str.strip()
sr_df.columns = sr_df.iloc[0].str.strip()
sheet_26as_df = sheet_26as_df.drop(0).reset_index(drop=True)
sr_df = sr_df.drop(0).reset_index(drop=True)
print("Cleaned 26as sheet columns:", sheet_26as_df.columns)
print("Cleaned Sr sheet columns:", sr_df.columns)

Cleaned 26as sheet columns: Index([                      'S',               'Cust_name',
                       'TAN_num',                     'Sec',
                        'T_date',                       nan,
                        'B_date',               'Total_amt',
                  'TDS_Deducted',           'TDS_Deposited',
       'Tracking Reference 26AS',                      'df',
                            'df',                      'df',
                             's',                       's',
                             's',                       's',
                             's',                       's',
                             nan,                       nan,
                             nan,                       nan,
                             nan,                      'SR',
                             nan,                       nan,
                             nan,                       nan],
      dtype='object', name=0)
Cleaned Sr sheet columns: 

In [18]:
df_26as = sheet_26as_df.loc[:, ['Cust_name', 'TAN_num', 'Sec', 'T_date', 'B_date', 'Total_amt', 'TDS_Deducted']]
df_26as.columns = ['Customer Name', 'TAN Number', 'Section', 'Transaction Date', 'Base Date', 'Total Amount', 'TDS Deducted']

In [19]:
df_sr = sr_df.loc[:, ['Customer Name', 'TAN Number', 'Taxable Value', 'TDS Amount', 'Posting Date', 'Internal Code']]
df_sr.columns = ['Customer Name', 'TAN Number', 'Taxable Value', 'TDS Amount', 'Posting Date', 'Internal Code']

KeyError: "None of [Index(['Customer Name', 'TAN Number', 'Taxable Value', 'TDS Amount',\n       'Posting Date', 'Internal Code'],\n      dtype='object', name=0)] are in the [columns]"

In [20]:
#if by any chance the data is missing I also included concepts of Ml here (simple imputer we are using here)
imputer = SimpleImputer(strategy='most_frequent')
df_26as = pd.DataFrame(imputer.fit_transform(df_26as), columns=df_26as.columns)
df_sr = pd.DataFrame(imputer.fit_transform(df_sr), columns=df_sr.columns)


NameError: name 'df_sr' is not defined

In [21]:
df_26as['Transaction Date'] = pd.to_datetime(df_26as['Transaction Date'], errors='coerce')
df_26as['Base Date'] = pd.to_datetime(df_26as['Base Date'], errors='coerce')
df_sr['Posting Date'] = pd.to_datetime(df_sr['Posting Date'], errors='coerce')
df_26as['End Date'] = df_26as['Base Date'] - pd.Timedelta(days=90)

NameError: name 'df_sr' is not defined

In [22]:
merged_df = pd.merge(df_26as, df_sr, on='Customer Name', suffixes=('_26as', '_Sr'))
merged_df['Amount_Match'] = (merged_df['Total Amount'] == merged_df['Taxable Value']).astype(int)
merged_df['TDS_Match'] = (merged_df['TDS Deducted'] == merged_df['TDS Amount']).astype(int)
merged_df['Date_Match_1'] = (merged_df['Transaction Date'] >= merged_df['Posting Date']).astype(int)
merged_df['Date_Match_2'] = (merged_df['Base Date'] >= merged_df['End Date']).astype(int)
merged_df['Match'] = (
    (merged_df['Amount_Match'] == 1) &
    (merged_df['TDS_Match'] == 1) &
    (merged_df['Date_Match_1'] == 1) &
    (merged_df['Date_Match_2'] == 1)
).astype(int)


NameError: name 'df_sr' is not defined

In [23]:
matched_transactions = merged_df[merged_df['Match'] == 1]
match_summary = matched_transactions.groupby('Internal Code').size().reset_index(name='Match Count')
match_summary['Match Type'] = 'BULK ABSOLUTE - Match 5a'
match_summary.to_csv('matched_transactions_summary.csv', index=False)
print(match_summary)


NameError: name 'merged_df' is not defined